In [1]:
# Standard library imports
import os
import sys
import json
import uuid

# Third-party imports
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import skimage.draw

# Custom module import
sys.path.append("../")  # Add parent directory to Python path
import LLMP as L

# ------------------- Configuration -------------------

# Main output directory
main_output_dir = "outputexp2"

# Subdirectories for images and JSON files
image_output_dir = os.path.join(main_output_dir, "images")
json_output_dir = os.path.join(main_output_dir, "json")

# Create directories if they don't exist
os.makedirs(image_output_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)

# Define tasks with associated questions
tasks = {
"bar": "Estimate the percentages of the four smaller bars relative to the largest (marked '.'). Each value is between 3 and 39, totaling 100. Express each proportion as a decimal between 0 and 1, with the largest bar as 1. Format: [1, 0.00, 0.00, 0.00, 0.00], starting from the marked bar, left to right. No explanation.",

"pie": "Estimate the percentages of the four smaller segments relative to the largest (marked '.'). Each value is between 3 and 39, totaling 100. Express each proportion as a decimal between 0 and 1, with the largest segment as 1. Format: [1, 0.00, 0.00, 0.00, 0.00], starting from the marked segment, counter-clockwise. No explanation."

}

# Number of images to generate for each task
num_images_per_task = 5

# List to store all data from all tasks
combined_dataset = []

# Loop through each task
for task, question in tasks.items():
    print(f"Generating images and dataset for task: {task}")
    
    # Set up a loop to generate images and collect their labels
    for i in range(num_images_per_task):
        # Generate the image and label for the task using GPImage
        image_array, label = L.GPImage.figure3(task)  # Ensure GPImage is defined or imported

        # Convert labels to Python-native floats for JSON compatibility
        label = [round(float(val), 2) for val in label]
        
        # Convert the array to uint8 format (values from 0 to 255) for saving as an image
        image_array_uint8 = (image_array * 255).astype(np.uint8)

        # Convert the NumPy array to a PIL image
        pil_image = Image.fromarray(image_array_uint8)

        # Generate a unique ID for the image
        unique_id = str(uuid.uuid4())

        # Save the image with the unique ID
        image_filename = os.path.join(image_output_dir, f"{unique_id}.jpg")
        pil_image.save(image_filename)

        # Create a JSON entry for the dataset
        json_entry = {
            'id': unique_id,
            'image': f"{unique_id}.jpg",
            'question': question,
            'value': label  # Normalized label values as native Python floats
        }

        # Append the JSON entry to the combined dataset list
        combined_dataset.append(json_entry)

# Save the combined dataset as a single JSON file in the JSON folder
combined_json_filename = "combined_dataset.json"
combined_json_filepath = os.path.join(json_output_dir, combined_json_filename)

with open(combined_json_filepath, 'w') as json_file:
    json.dump(combined_dataset, json_file, indent=4)

print(f"Images saved in '{image_output_dir}' and combined dataset saved as '{combined_json_filename}' in '{json_output_dir}'")


Generating images and dataset for task: bar
Generated Labels: [1.   0.21 0.04 0.18 0.24]
Generated Labels: [1.   0.15 0.12 0.26 0.11]
Generated Labels: [1.   0.2  0.1  0.19 0.19]
Generated Labels: [1.   0.12 0.15 0.19 0.14]
Generated Labels: [1.   0.22 0.06 0.12 0.26]
Generating images and dataset for task: pie
Generated Labels: [1.   0.08 0.23 0.14 0.22]
Generated Labels: [1.   0.36 0.1  0.07 0.05]
Generated Labels: [1.   0.08 0.11 0.17 0.24]
Generated Labels: [1.   0.2  0.21 0.1  0.12]
Generated Labels: [1.   0.08 0.29 0.18 0.13]
Images saved in 'outputexp2/images' and combined dataset saved as 'combined_dataset.json' in 'outputexp2/json'


### Distribution for each task 


### Count each unique label for each task
